In [23]:
%pip install Unidecode
%pip install num2words
%pip install spacy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/11.8 MB ? eta -:--:--
   ------------------ --------------------- 5.5/11.8 MB 25.8 MB/s eta 0:00:01
   -------------------------------------- - 11.3/11.8 MB 27.1 MB/s eta 0:00:01
   ---------------------------------------- 11.8/11.8 MB 24.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/632.6 kB ? eta -:--:--
   --------------------------------------- 632.6/632.6 kB 12.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 19.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   -------------------------------------- - 6.0/6.3 MB 30.7 MB/s eta 0:00:01
   ---------------------------------------- 6.3/6.3 MB 27.4 MB/s eta 0:00:00
   ---------------------------------------

In [ ]:
import pandas as pd
import re
from unidecode import unidecode
import num2words
import spacy
from spacy.lang.es import stop_words as stop_words

In [14]:
df=pd.read_csv('fake_news_spanish.csv', encoding='utf-8', delimiter=';')
df

,ID,Label,Titulo,Descripcion,Fecha
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018
...,...,...,...,...,...
57058,ID,1,El Defensor del Pueblo reclama a la Comunidad ...,El gobierno regional han indicado que la atenc...,08/06/2021
57059,ID,0,El EQUO plantea ceder la presidencia de la Com...,Si la higiene democrática nos lleva a esa exig...,08/09/2020
57060,ID,1,Alberto Garzón: 'Que los Borbones son unos lad...,El coordinador federal de IU asegura que la mo...,12/07/2018
57061,ID,1,Vox exige entrar en el Gobierno de Castilla y ...,Santiago Abascal: Vox tiene el derecho y el de...,13/02/2022


In [15]:
df['Titulo']=df['Titulo'].fillna('')
df['d']=df['Titulo']+df['Descripcion']
df


,ID,Label,Titulo,Descripcion,Fecha,d
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023,'The Guardian' va con Sánchez: 'Europa necesit...
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022,El 'Ahora o nunca' de Joan Fuster sobre el est...
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild..."
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018,Puigdemont: 'No sería ninguna tragedia una rep...
...,...,...,...,...,...,...
57058,ID,1,El Defensor del Pueblo reclama a la Comunidad ...,El gobierno regional han indicado que la atenc...,08/06/2021,El Defensor del Pueblo reclama a la Comunidad ...
57059,ID,0,El EQUO plantea ceder la presidencia de la Com...,Si la higiene democrática nos lleva a esa exig...,08/09/2020,El EQUO plantea ceder la presidencia de la Com...
57060,ID,1,Alberto Garzón: 'Que los Borbones son unos lad...,El coordinador federal de IU asegura que la mo...,12/07/2018,Alberto Garzón: 'Que los Borbones son unos lad...
57061,ID,1,Vox exige entrar en el Gobierno de Castilla y ...,Santiago Abascal: Vox tiene el derecho y el de...,13/02/2022,Vox exige entrar en el Gobierno de Castilla y ...


In [16]:
def normalizar(texto):
    # todo a minúsculas
    texto = texto.lower()

    # tildes y diacríticas
    texto = unidecode(texto)

    return texto


In [17]:
df['d']=df['d'].apply(normalizar)
df


,ID,Label,Titulo,Descripcion,Fecha,d
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023,'the guardian' va con sanchez: 'europa necesit...
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023,revelan que el gobierno negocio la liberacion ...
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022,el 'ahora o nunca' de joan fuster sobre el est...
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022,"iglesias alienta a yolanda diaz, erc y eh bild..."
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018,puigdemont: 'no seria ninguna tragedia una rep...
...,...,...,...,...,...,...
57058,ID,1,El Defensor del Pueblo reclama a la Comunidad ...,El gobierno regional han indicado que la atenc...,08/06/2021,el defensor del pueblo reclama a la comunidad ...
57059,ID,0,El EQUO plantea ceder la presidencia de la Com...,Si la higiene democrática nos lleva a esa exig...,08/09/2020,el equo plantea ceder la presidencia de la com...
57060,ID,1,Alberto Garzón: 'Que los Borbones son unos lad...,El coordinador federal de IU asegura que la mo...,12/07/2018,alberto garzon: 'que los borbones son unos lad...
57061,ID,1,Vox exige entrar en el Gobierno de Castilla y ...,Santiago Abascal: Vox tiene el derecho y el de...,13/02/2022,vox exige entrar en el gobierno de castilla y ...


In [18]:
def tokenizar(texto):
    # dividir el texto en palabras
    palabras = re.findall(r'\b\w+\b', texto)
    return palabras

df['d']=df['d'].apply(tokenizar)
df
df['d']

0        [the, guardian, va, con, sanchez, europa, nece...
1        [revelan, que, el, gobierno, negocio, la, libe...
2        [el, ahora, o, nunca, de, joan, fuster, sobre,...
3        [iglesias, alienta, a, yolanda, diaz, erc, y, ...
4        [puigdemont, no, seria, ninguna, tragedia, una...
                               ...                        
57058    [el, defensor, del, pueblo, reclama, a, la, co...
57059    [el, equo, plantea, ceder, la, presidencia, de...
57060    [alberto, garzon, que, los, borbones, son, uno...
57061    [vox, exige, entrar, en, el, gobierno, de, cas...
57062    [unas, 300, personas, protestan, contra, la, v...
Name: d, Length: 57063, dtype: object

In [20]:
def num_to_word(line):
    new_line=[]
    x=r'\d'
    y=r'[a-zA-Z]'
    for word in line:
        if re.search(x,word) and not re.search(y,word):
            word=float(word)
            new_line.append(num2words.num2words(word, lang = 'es_CO'))
        else:
            new_line.append(word)
    return new_line

#tricentésimo
df['d']=df['d'].apply(num_to_word)
df

,ID,Label,Titulo,Descripcion,Fecha,d
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023,"[the, guardian, va, con, sanchez, europa, nece..."
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023,"[revelan, que, el, gobierno, negocio, la, libe..."
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022,"[el, ahora, o, nunca, de, joan, fuster, sobre,..."
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022,"[iglesias, alienta, a, yolanda, diaz, erc, y, ..."
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018,"[puigdemont, no, seria, ninguna, tragedia, una..."
...,...,...,...,...,...,...
57058,ID,1,El Defensor del Pueblo reclama a la Comunidad ...,El gobierno regional han indicado que la atenc...,08/06/2021,"[el, defensor, del, pueblo, reclama, a, la, co..."
57059,ID,0,El EQUO plantea ceder la presidencia de la Com...,Si la higiene democrática nos lleva a esa exig...,08/09/2020,"[el, equo, plantea, ceder, la, presidencia, de..."
57060,ID,1,Alberto Garzón: 'Que los Borbones son unos lad...,El coordinador federal de IU asegura que la mo...,12/07/2018,"[alberto, garzon, que, los, borbones, son, uno..."
57061,ID,1,Vox exige entrar en el Gobierno de Castilla y ...,Santiago Abascal: Vox tiene el derecho y el de...,13/02/2022,"[vox, exige, entrar, en, el, gobierno, de, cas..."


In [ ]:
stopwords_espanol = stop_words.STOP_WORDS
#Se usa Spacy porque tiene un "vocabulario" más amplio que NLTK
#Genera un mejor resultado debido a esto

def stopwords_espanol_remove(line):
    new_line=[]
    for word in line:
        if word not in stopwords_espanol:
            new_line.append(word)
    return new_line


df['d']=df['d'].apply(stopwords_espanol_remove)
df

,ID,Label,Titulo,Descripcion,Fecha,d
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023,"[the, guardian, sanchez, europa, necesita, apu..."
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023,"[revelan, gobierno, negocio, liberacion, mirel..."
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022,"[joan, fuster, estatuto, valenciano, cumple, c..."
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022,"[iglesias, alienta, yolanda, diaz, erc, eh, bi..."
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018,"[puigdemont, seria, tragedia, repeticion, elec..."
...,...,...,...,...,...,...
57058,ID,1,El Defensor del Pueblo reclama a la Comunidad ...,El gobierno regional han indicado que la atenc...,08/06/2021,"[defensor, pueblo, reclama, comunidad, madrid,..."
57059,ID,0,El EQUO plantea ceder la presidencia de la Com...,Si la higiene democrática nos lleva a esa exig...,08/09/2020,"[equo, plantea, ceder, presidencia, comunidad,..."
57060,ID,1,Alberto Garzón: 'Que los Borbones son unos lad...,El coordinador federal de IU asegura que la mo...,12/07/2018,"[alberto, garzon, borbones, ladrones, historic..."
57061,ID,1,Vox exige entrar en el Gobierno de Castilla y ...,Santiago Abascal: Vox tiene el derecho y el de...,13/02/2022,"[vox, exige, entrar, gobierno, castilla, leon,..."
